In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, ElementNotInteractableException
import random
from time import sleep
from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore", category=UserWarning) 

In [2]:
driver = webdriver.Firefox(executable_path='gd/geckodriver.exe')
driver.get("http://www.wix.com/signin")
driver.find_element_by_id('input_0').send_keys("REPLACE WITH YOUR LOGIN EMAIL")
driver.find_element_by_id('input_1').send_keys('REPLACE WITH YOUR LOGIN PASSWORD')

ElementNotInteractableException: Message: Element <input id="input_1" class="ng-pristine ng-untouched ng-scope md-input ng-empty ng-invalid ng-invalid-required ng-valid-minlength" name="password" type="password"> is not reachable by keyboard
Stacktrace:
WebDriverError@chrome://remote/content/shared/webdriver/Errors.jsm:183:5
ElementNotInteractableError@chrome://remote/content/shared/webdriver/Errors.jsm:293:5
webdriverSendKeysToElement@chrome://remote/content/marionette/interaction.js:624:13
interaction.sendKeysToElement@chrome://remote/content/marionette/interaction.js:600:11
sendKeysToElement@chrome://remote/content/marionette/actors/MarionetteCommandsChild.jsm:497:24
receiveMessage@chrome://remote/content/marionette/actors/MarionetteCommandsChild.jsm:151:31


Log in

Navigate to the collection and scroll to the bottom, making sure all of the products in the collection are visible.

In [48]:
def get_radio_box(prod_name):
    # finds the radio box associated with the product name in case of duplicates
    # prod_box = driver.find_elements_by_class_name('wix-list-item')
    prod_box = driver.find_elements_by_class_name('popover-element')
    for i in prod_box:
        try:
            products_name = str(i.find_element_by_class_name('popover-element').text).replace('’', '').lower()
            if str(prod_name).lower() in products_name:
                return i.find_element_by_class_name('checkbox-box')
        except: 
            pass

In [46]:
def add_to_collection(prod_name, collection_name):
    # adds a Product to a given collection
    prod_name = str(prod_name).replace('’', '').lower()
    try:
        # driver.find_element_by_class_name('wix-search-clear').click()
        driver.find_element_by_class_name('s3QYfwo').click()
    except:
        print('1')
        pass
    sleep(1)
    try:
        WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "s3cite5"))).click()
    except TimeoutException:
        print('2')
        driver.refresh()
        sleep(60)
        WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "s3site5"))).click()
    sleep(1)
    if len(prod_name)>21:
        send_str = str(prod_name).lower()[-21:]
    else:
        send_str = str(prod_name).lower()
    driver.find_element_by_id('s3cite').send_keys(send_str)
    sleep(5)
    x = 0
    while (len(driver.find_elements_by_class_name('wix-checkbox-inner')) < 2) and (x<10):
        sleep(1)
        x+=1
    prod_radio = get_radio_box(prod_name)
    if (x != 10) and prod_radio is not None:
        prod_radio.click()
        sleep(1)
        driver.find_element_by_xpath('/html/body/div[1]/div/div/div[2]/div/div[1]/div/div/div/div/div/store-manager/div/main/products-list/page-header/div[2]/ul/li[1]/list-header/div[2]/dropdown-button[1]/button/span').click()
        sleep(1)
        driver.find_elements_by_class_name('wix-search-wrapper')[1].click()
        for i in driver.find_elements_by_class_name('ng-pristine'):
            try:
                i.send_keys(collection_name)
            except:
                print('3')
                pass
        sleep(1)
        coll = driver.find_element_by_class_name('name')
        assert(str(coll.text).lower() == str(collection_name).lower())
        coll.click()
        driver.find_elements_by_xpath("//*[@data-hook='apply']")[0].click()
    else:
        print('couldnt find {}'.format(prod_name))
    sleep(5)

In [37]:
def add_all():
    # Get collection details and shuffle Product list
    prods = driver.find_elements_by_class_name('product-name')
    removes = driver.find_elements_by_class_name('remove-button')
    print(len(prods))
    prod_names = [x.text for x in prods]
    # collection_name = driver.find_element_by_class_name('label-text').text

    prod_list = list(zip(range(len(prod_names)), prod_names, removes))
    random.shuffle(prod_list)
    
    # return [x for x in prod_list]
    return prod_names

In [38]:
def get_missing(original_collection, shuffled_collection):
    return [elem for elem in original_collection if elem not in shuffled_collection]

# Randomizing Collections

Go to collection you wish to randomize and scroll to the last product.

In [41]:
prod_list = add_all()

534


Create new collection and enter its name below

In [42]:
collection_name = 'all_products_20221126'

In [13]:
# prod_list.remove('Cats Playing Hippos 1000 Piece Puzzle')

In [15]:
# prod_list_end = prod_list[100:]

In [43]:
prod_list[:5]

['Cats Vs Pickles Huggers Plush Toy - Taco Cat',
 'Cats Vs Pickles Huggers Plush Toy - Triple Scoop',
 'Cats Vs Pickles Huggers Plush Toy - Rainbow Cake',
 'Cats Vs Pickles Huggers Plush Toy - Comet Hartley 2',
 'Cats Vs Pickles Huggers Plush Toy - Llama']

In [64]:
driver.find_elements_by_xpath('/html/body/div[2]/div/div[2]/div[2]/div[2]/div[1]/div/div/div/div/span/div[1]/div/div/div/div[2]/div/div/div/div/div[2]/div[4]/div/div/div/table/tbody/tr[2]/td[3]/a/div/div/div/span')[0].text

'Cats Vs Pickles Huggers Plush Toy - Triple Scoop'

In [75]:
# driver.find_elements_by_css_selector('ProductListPage_product_name_1')
# driver.find_elements_by_name('Cats Vs Pickles Huggers Plush Toy - Triple Scoop')
# driver .find_elements_by_class_name('ProductListPage-product-name-1')
element = "tr.s1-01wE:nth-child(1) > td:nth-child(3) > a:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > span:nth-child(1)"
driver.find_element_by_css_selector(element).text
# tr.s1-01wE:nth-child(2) > td:nth-child(3) > a:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > span:nth-child(1)
# tr.s1-01wE:nth-child(1) > td:nth-child(3) > a:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > span:nth-child(1)

'Cats Vs Pickles Huggers Plush Toy - Taco Cat'

In [81]:
element = 'tr.s1-01wE:nth-child(7) > td:nth-child(1) > div:nth-child(1) > label:nth-child(2) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1)'

In [ ]:
tr.s1-01wE:nth-child(1) > td:nth-child(1) > div:nth-child(1) > label:nth-child(2) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1)
tr.s1-01wE:nth-child(2) > td:nth-child(1) > div:nth-child(1) > label:nth-child(2) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1)

In [88]:
radios = []
i = 1
while True:
    try:
        element = 'tr.s1-01wE:nth-child({}) > td:nth-child(1) > div:nth-child(1) > label:nth-child(2) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1)'.format(i)
        radio_button = driver.find_element_by_css_selector(element)
        radios.append(radio_button)
        i+=1
        print(i)
        #code with possible error
    except:
         continue
    else:
         #the rest of the code
         break

2


In [87]:
radios[0].click()

In [82]:
driver.find_element_by_css_selector(element).click()

NoSuchElementException: Message: Unable to locate element: tr.s1-01wE:nth-child(7) > td:nth-child(1) > div:nth-child(1) > label:nth-child(2) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1)
Stacktrace:
WebDriverError@chrome://remote/content/shared/webdriver/Errors.jsm:183:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.jsm:395:5
element.find/</<@chrome://remote/content/marionette/element.js:300:16


In [47]:
add_to_collection(prod_list[0], collection_name)

1
2



KeyboardInterrupt



go to products to automate add to collection service

In [44]:
y = 0
for i in tqdm(prod_list):
    if y % 20 == 0:
        driver.refresh()
        sleep(10)
    add_to_collection(i, collection_name)
    y+=1

  0%|                                                                                          | 0/534 [01:48<?, ?it/s]

KeyboardInterrupt



In [12]:
prod_list[100]

'Cats Playing Hippos 1000 Piece Puzzle'

# Checking for missing products in catelogs

Go to new collection and scroll to bottom

In [22]:
shuf_col, _ = add_all()

149


In [23]:
missing = get_missing(prod_list, shuf_col)

Either re-run the add code block using the new missing collection or just add manually. Normally some manual adding is needed, around 4-5 products on average.

In [24]:
missing

["Men's Spooky Cat Socks",
 'Men’s Chase Crew Socks',
 'Knittin’ Kitten Enamel Pin',
 'Men’s Hipster Cat Socks',
 'Men’s Angel Cat Socks',
 'Men’s Wild Tiger Socks',
 'Men’s Tiger Crew Socks',
 'Bookish Cats 500 Piece Puzzle']

In [21]:
y = 0
for i in tqdm(missing):
    if y % 20 == 0:
        driver.refresh()
        sleep(10)
    add_to_collection(i, collection_name)
    y+=1

 14%|███████████▍                                                                       | 4/29 [01:30<08:22, 20.11s/it]

couldnt find mens chase crew socks


 17%|██████████████▎                                                                    | 5/29 [01:53<08:26, 21.09s/it]

couldnt find knittin kitten enamel pin


 28%|██████████████████████▉                                                            | 8/29 [02:49<06:41, 19.10s/it]

...

 90%|█████████████████████████████████████████████████████████████████████████▌        | 26/29 [08:31<00:55, 18.43s/it]

couldnt find mens tiger crew socks


 93%|████████████████████████████████████████████████████████████████████████████▎     | 27/29 [08:54<00:39, 19.77s/it]

couldnt find bookish cats 500 piece puzzle


100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [09:24<00:00, 19.46s/it]
